# Project title: Price model for airbnb listings in Geneva

## Data collection + preparing and storing data

## Libraries and settings

In [53]:
# Libraries
import os
import json
import numpy as np
import pandas as pd
from tabulate import tabulate
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

# API credentials for Kaggle
with open('kaggle.json') as f:
    data = json.load(f)

os.environ['KAGGLE_USERNAME'] = data['username']
os.environ['KAGGLE_KEY'] = data['key']

from kaggle.api.kaggle_api_extended import KaggleApi

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

/workspaces/project_arm_airbnb/data


## Use Kaggle Web API to download airbnb data for Geneva

In [54]:
# Initialize API
api = KaggleApi()
api.authenticate()

# Download file
api.dataset_download_file('prayankkul/airbnb-geneva-switzerland-29-jun-2021', path='.', file_name='listings_1.csv')

Dataset URL: https://www.kaggle.com/datasets/prayankkul/airbnb-geneva-switzerland-29-jun-2021


True

In [55]:
# Read zip file data to pandas data frame
# path to the zip file
zip_file_path = '/workspaces/project_arm_airbnb/data/listings_1.csv.zip'

# Entpacken der ZIP-Datei
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/workspaces/project_arm_airbnb/data')

# Lesen der entpackten CSV-Datei in ein Pandas DataFrame
csv_file_path = '/workspaces/project_arm_airbnb/data/listings_1.csv'
df_orig = pd.read_csv(csv_file_path)

# Anzeigen des DataFrames
df_orig.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,42515,https://www.airbnb.com/rooms/42515,20210629025133,2021-06-29,Geneva center - Hosting females,<b>The space</b><br />This is a private room w...,NaN,https://a0.muscache.com/pictures/10640277/ff1d...,185647,https://www.airbnb.com/users/show/185647,...,4.78,4.79,4.40,NaN,f,1,0,1,0,1.20
1,44959,https://www.airbnb.com/rooms/44959,20210629025133,2021-06-29,Perfect Cozy Bedroom Close to Lake & Transport,Perfect for the single traveler whether it be ...,The apartment is situated in the heart of Gene...,https://a0.muscache.com/pictures/3340619/139f4...,198343,https://www.airbnb.com/users/show/198343,...,5.00,4.72,4.57,NaN,f,1,0,1,0,0.67
2,194110,https://www.airbnb.com/rooms/194110,20210629025133,2021-06-29,Central apartment by the lake,Design apartment conveniently located in the h...,NaN,https://a0.muscache.com/pictures/1252014/0ee91...,942248,https://www.airbnb.com/users/show/942248,...,4.49,4.85,4.49,NaN,f,1,1,0,0,0.67
3,276025,https://www.airbnb.com/rooms/276025,20210629025133,2021-06-29,Lovely flat by Geneva lake,This cozy flat is located in a very pleasant v...,NaN,https://a0.muscache.com/pictures/4156456/d3aed...,1442438,https://www.airbnb.com/users/show/1442438,...,4.87,4.75,4.68,NaN,t,1,1,0,0,1.15
4,325986,https://www.airbnb.com/rooms/325986,20210629025133,2021-06-29,"Stylish loft in Pâquis, next to UN",Hello traveler!<br />Pâquis is one of the most...,NaN,https://a0.muscache.com/pictures/21414234/10b5...,1667362,https://www.airbnb.com/users/show/1667362,...,4.93,4.76,4.65,NaN,f,1,1,0,0,1.01


In [56]:
# delete zip file
if os.path.exists(zip_file_path):
    os.remove(zip_file_path)
    print(f"{zip_file_path} wurde gelöscht.")
else:
    print(f"{zip_file_path} existiert nicht.")

/workspaces/project_arm_airbnb/data/listings_1.csv.zip wurde gelöscht.


# Data preparation

In [57]:
# create a subset of the data with the columns we are interested in for the analysis
# select columns: 'id', 'neighbourhood_cleansed', 'latitude', 'longitude','property_type', 'room_type', 'price', 'accomodates', 'bathrooms_text', 'amenities', 'review_scores_rating', 'review_scores_location', 'instant_bookable', 'calculated_host_listings_count'
df = df_orig[['id', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'price', 'accommodates', 'bathrooms_text', 'amenities', 'review_scores_rating', 'review_scores_location', 'instant_bookable', 'calculated_host_listings_count']]
df.head()

,id,neighbourhood_cleansed,latitude,longitude,property_type,room_type,price,accommodates,bathrooms_text,amenities,review_scores_rating,review_scores_location,instant_bookable,calculated_host_listings_count
0,42515,Commune de Genève,46.20140,6.15507,Private room in apartment,Private room,$72.00,1,1.5 baths,"[""Cable TV"", ""Dishes and silverware"", ""Washer""...",4.69,4.79,f,1
1,44959,Commune de Genève,46.19704,6.16076,Private room in apartment,Private room,$68.00,1,1.5 shared baths,"[""Extra pillows and blankets"", ""Cable TV"", ""Po...",4.87,4.72,f,1
2,194110,Commune de Genève,46.20420,6.16067,Entire apartment,Entire home/apt,$135.00,4,1 bath,"[""Extra pillows and blankets"", ""Cable TV"", ""Di...",4.50,4.85,f,1
3,276025,Versoix,46.27944,6.16724,Entire apartment,Entire home/apt,$65.00,5,1 bath,"[""Extra pillows and blankets"", ""Portable fans""...",4.62,4.75,t,1
4,325986,Commune de Genève,46.21428,6.14610,Entire apartment,Entire home/apt,$150.00,2,1.5 baths,"[""Dishes and silverware"", ""Washer"", ""TV"", ""Smo...",4.84,4.76,f,1


In [58]:
# print shape of the data
print(f"Dataframe shape: {df.shape}")

# print number of columns and rows
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")


Dataframe shape: (2069, 14)
Number of rows: 2069
Number of columns: 14


### Check for hotels in data

In [59]:
# unique values of the property_type column
property_types = df['property_type'].unique()
print(property_types)

# count the number of listings for each property type
property_type_counts = df['property_type'].value_counts()
print(property_type_counts)

['Private room in apartment' 'Entire apartment' 'Private room in loft'
 'Private room in condominium' 'Entire loft' 'Private room in villa'
 'Entire condominium' 'Private room in house' 'Entire house'
 'Entire villa' 'Private room in bed and breakfast' 'Entire guesthouse'
 'Entire place' 'Entire bungalow' 'Shared room in loft'
 'Entire serviced apartment' 'Shared room in apartment'
 'Entire guest suite' 'Room in hotel' 'Entire bed and breakfast'
 'Private room in nature lodge' 'Private room in townhouse'
 'Private room in casa particular' 'Room in boutique hotel'
 'Entire townhouse' 'Tiny house' 'Shared room'
 'Private room in guesthouse' 'Yurt' 'Private room in hostel' 'Boat'
 'Private room in chalet' 'Farm stay' 'Shared room in condominium'
 'Room in aparthotel' 'Private room in guest suite' 'Camper/RV'
 'Private room in tiny house' 'Houseboat' 'Bus'
 'Private room in serviced apartment' 'Private room' 'Entire cottage']
property_type
Entire apartment                      1084
Private

In [60]:
# Only include property types of private indivduals and exclude professional / commercial property types 
# exclude property types with hotel or hostel in the name
df = df[~df['property_type'].str.contains('Hotel|Hostel|hotel|hostel')]
df['property_type'].value_counts()

property_type
Entire apartment                      1084
Private room in apartment              435
Entire serviced apartment              111
Entire condominium                      89
Private room in house                   71
Entire house                            41
Private room in condominium             31
Entire loft                             19
Private room in bed and breakfast       19
Private room in villa                   16
Private room in townhouse               12
Shared room in apartment                11
Private room in casa particular         10
Entire villa                            10
Private room in loft                     9
Entire guesthouse                        9
Entire townhouse                         7
Boat                                     6
Entire guest suite                       5
Private room in serviced apartment       5
Tiny house                               3
Farm stay                                2
Private room in nature lodge            

In [61]:
# print shape of the data
print(f"Dataframe shape: {df.shape}")

# print number of columns and rows
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

Dataframe shape: (2024, 14)
Number of rows: 2024
Number of columns: 14


### Conversion of variables 'price' and 'bathrooms' to floats

In [62]:
# convert price to float
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)
df.head()

,id,neighbourhood_cleansed,latitude,longitude,property_type,room_type,price,accommodates,bathrooms_text,amenities,review_scores_rating,review_scores_location,instant_bookable,calculated_host_listings_count
0,42515,Commune de Genève,46.20140,6.15507,Private room in apartment,Private room,72.0,1,1.5 baths,"[""Cable TV"", ""Dishes and silverware"", ""Washer""...",4.69,4.79,f,1
1,44959,Commune de Genève,46.19704,6.16076,Private room in apartment,Private room,68.0,1,1.5 shared baths,"[""Extra pillows and blankets"", ""Cable TV"", ""Po...",4.87,4.72,f,1
2,194110,Commune de Genève,46.20420,6.16067,Entire apartment,Entire home/apt,135.0,4,1 bath,"[""Extra pillows and blankets"", ""Cable TV"", ""Di...",4.50,4.85,f,1
3,276025,Versoix,46.27944,6.16724,Entire apartment,Entire home/apt,65.0,5,1 bath,"[""Extra pillows and blankets"", ""Portable fans""...",4.62,4.75,t,1
4,325986,Commune de Genève,46.21428,6.14610,Entire apartment,Entire home/apt,150.0,2,1.5 baths,"[""Dishes and silverware"", ""Washer"", ""TV"", ""Smo...",4.84,4.76,f,1


In [63]:
# drop in bathrooms_text all text that starts with b or s
df['bathrooms_text'] = df['bathrooms_text'].str.replace(r'[^0-9.]+', '', regex=True)
df.head()

,id,neighbourhood_cleansed,latitude,longitude,property_type,room_type,price,accommodates,bathrooms_text,amenities,review_scores_rating,review_scores_location,instant_bookable,calculated_host_listings_count
0,42515,Commune de Genève,46.20140,6.15507,Private room in apartment,Private room,72.0,1,1.5,"[""Cable TV"", ""Dishes and silverware"", ""Washer""...",4.69,4.79,f,1
1,44959,Commune de Genève,46.19704,6.16076,Private room in apartment,Private room,68.0,1,1.5,"[""Extra pillows and blankets"", ""Cable TV"", ""Po...",4.87,4.72,f,1
2,194110,Commune de Genève,46.20420,6.16067,Entire apartment,Entire home/apt,135.0,4,1,"[""Extra pillows and blankets"", ""Cable TV"", ""Di...",4.50,4.85,f,1
3,276025,Versoix,46.27944,6.16724,Entire apartment,Entire home/apt,65.0,5,1,"[""Extra pillows and blankets"", ""Portable fans""...",4.62,4.75,t,1
4,325986,Commune de Genève,46.21428,6.14610,Entire apartment,Entire home/apt,150.0,2,1.5,"[""Dishes and silverware"", ""Washer"", ""TV"", ""Smo...",4.84,4.76,f,1


In [64]:
# convert bathrooms_text to float and set 1 for NaN and missing values
df['bathrooms_text'] = df['bathrooms_text'].replace('', np.nan).fillna(1).astype(float)
df.head()

,id,neighbourhood_cleansed,latitude,longitude,property_type,room_type,price,accommodates,bathrooms_text,amenities,review_scores_rating,review_scores_location,instant_bookable,calculated_host_listings_count
0,42515,Commune de Genève,46.20140,6.15507,Private room in apartment,Private room,72.0,1,1.5,"[""Cable TV"", ""Dishes and silverware"", ""Washer""...",4.69,4.79,f,1
1,44959,Commune de Genève,46.19704,6.16076,Private room in apartment,Private room,68.0,1,1.5,"[""Extra pillows and blankets"", ""Cable TV"", ""Po...",4.87,4.72,f,1
2,194110,Commune de Genève,46.20420,6.16067,Entire apartment,Entire home/apt,135.0,4,1.0,"[""Extra pillows and blankets"", ""Cable TV"", ""Di...",4.50,4.85,f,1
3,276025,Versoix,46.27944,6.16724,Entire apartment,Entire home/apt,65.0,5,1.0,"[""Extra pillows and blankets"", ""Portable fans""...",4.62,4.75,t,1
4,325986,Commune de Genève,46.21428,6.14610,Entire apartment,Entire home/apt,150.0,2,1.5,"[""Dishes and silverware"", ""Washer"", ""TV"", ""Smo...",4.84,4.76,f,1


In [65]:
# rename bathrooms_text to bathrooms
df.rename(columns={'bathrooms_text': 'bathrooms'}, inplace=True)

# rename latitude to lat and longitude to lon
df.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)

df.head()

,id,neighbourhood_cleansed,lat,lon,property_type,room_type,price,accommodates,bathrooms,amenities,review_scores_rating,review_scores_location,instant_bookable,calculated_host_listings_count
0,42515,Commune de Genève,46.20140,6.15507,Private room in apartment,Private room,72.0,1,1.5,"[""Cable TV"", ""Dishes and silverware"", ""Washer""...",4.69,4.79,f,1
1,44959,Commune de Genève,46.19704,6.16076,Private room in apartment,Private room,68.0,1,1.5,"[""Extra pillows and blankets"", ""Cable TV"", ""Po...",4.87,4.72,f,1
2,194110,Commune de Genève,46.20420,6.16067,Entire apartment,Entire home/apt,135.0,4,1.0,"[""Extra pillows and blankets"", ""Cable TV"", ""Di...",4.50,4.85,f,1
3,276025,Versoix,46.27944,6.16724,Entire apartment,Entire home/apt,65.0,5,1.0,"[""Extra pillows and blankets"", ""Portable fans""...",4.62,4.75,t,1
4,325986,Commune de Genève,46.21428,6.14610,Entire apartment,Entire home/apt,150.0,2,1.5,"[""Dishes and silverware"", ""Washer"", ""TV"", ""Smo...",4.84,4.76,f,1


### Conversion of variable 'instant_bookable' to binary (0/1)

In [66]:
# convert vriable instant_bookable to dummy variable with 1 for True and 0 for False
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})
df.head()

,id,neighbourhood_cleansed,lat,lon,property_type,room_type,price,accommodates,bathrooms,amenities,review_scores_rating,review_scores_location,instant_bookable,calculated_host_listings_count
0,42515,Commune de Genève,46.20140,6.15507,Private room in apartment,Private room,72.0,1,1.5,"[""Cable TV"", ""Dishes and silverware"", ""Washer""...",4.69,4.79,0,1
1,44959,Commune de Genève,46.19704,6.16076,Private room in apartment,Private room,68.0,1,1.5,"[""Extra pillows and blankets"", ""Cable TV"", ""Po...",4.87,4.72,0,1
2,194110,Commune de Genève,46.20420,6.16067,Entire apartment,Entire home/apt,135.0,4,1.0,"[""Extra pillows and blankets"", ""Cable TV"", ""Di...",4.50,4.85,0,1
3,276025,Versoix,46.27944,6.16724,Entire apartment,Entire home/apt,65.0,5,1.0,"[""Extra pillows and blankets"", ""Portable fans""...",4.62,4.75,1,1
4,325986,Commune de Genève,46.21428,6.14610,Entire apartment,Entire home/apt,150.0,2,1.5,"[""Dishes and silverware"", ""Washer"", ""TV"", ""Smo...",4.84,4.76,0,1


### New variable 'lake_access' from amenities

In [67]:
# create new variable 'lake_access' with 1 for listings with lake access and 0 for listings without lake access
df['lake_access'] = df['amenities'].str.contains('Lake access').astype(int)
df.head()

,id,neighbourhood_cleansed,lat,lon,property_type,room_type,price,accommodates,bathrooms,amenities,review_scores_rating,review_scores_location,instant_bookable,calculated_host_listings_count,lake_access
0,42515,Commune de Genève,46.20140,6.15507,Private room in apartment,Private room,72.0,1,1.5,"[""Cable TV"", ""Dishes and silverware"", ""Washer""...",4.69,4.79,0,1,0
1,44959,Commune de Genève,46.19704,6.16076,Private room in apartment,Private room,68.0,1,1.5,"[""Extra pillows and blankets"", ""Cable TV"", ""Po...",4.87,4.72,0,1,0
2,194110,Commune de Genève,46.20420,6.16067,Entire apartment,Entire home/apt,135.0,4,1.0,"[""Extra pillows and blankets"", ""Cable TV"", ""Di...",4.50,4.85,0,1,0
3,276025,Versoix,46.27944,6.16724,Entire apartment,Entire home/apt,65.0,5,1.0,"[""Extra pillows and blankets"", ""Portable fans""...",4.62,4.75,1,1,1
4,325986,Commune de Genève,46.21428,6.14610,Entire apartment,Entire home/apt,150.0,2,1.5,"[""Dishes and silverware"", ""Washer"", ""TV"", ""Smo...",4.84,4.76,0,1,1


### New variable 'count_amenities'

In [68]:
# create a new variable num_amenities based on the number of amenities
df['num_amenities'] = df['amenities'].str.split(',').str.len()
df.head()

,id,neighbourhood_cleansed,lat,lon,property_type,room_type,price,accommodates,bathrooms,amenities,review_scores_rating,review_scores_location,instant_bookable,calculated_host_listings_count,lake_access,num_amenities
0,42515,Commune de Genève,46.20140,6.15507,Private room in apartment,Private room,72.0,1,1.5,"[""Cable TV"", ""Dishes and silverware"", ""Washer""...",4.69,4.79,0,1,0,22
1,44959,Commune de Genève,46.19704,6.16076,Private room in apartment,Private room,68.0,1,1.5,"[""Extra pillows and blankets"", ""Cable TV"", ""Po...",4.87,4.72,0,1,0,51
2,194110,Commune de Genève,46.20420,6.16067,Entire apartment,Entire home/apt,135.0,4,1.0,"[""Extra pillows and blankets"", ""Cable TV"", ""Di...",4.50,4.85,0,1,0,26
3,276025,Versoix,46.27944,6.16724,Entire apartment,Entire home/apt,65.0,5,1.0,"[""Extra pillows and blankets"", ""Portable fans""...",4.62,4.75,1,1,1,48
4,325986,Commune de Genève,46.21428,6.14610,Entire apartment,Entire home/apt,150.0,2,1.5,"[""Dishes and silverware"", ""Washer"", ""TV"", ""Smo...",4.84,4.76,0,1,1,26


### Summary statistics and missing values

In [69]:
# describe the data
# print the summary statistics of the data
print(df.describe())


                 id          lat          lon        price  accommodates  \
count  2.024000e+03  2024.000000  2024.000000  2024.000000   2024.000000   
mean   2.992165e+07    46.206284     6.143972   126.394269      2.657115   
std    1.529775e+07     0.020470     0.027914   134.113890      1.507280   
min    4.251500e+04    46.138820     5.973220    10.000000      1.000000   
25%    1.723789e+07    46.195377     6.132483    70.000000      2.000000   
50%    3.307640e+07    46.204440     6.144230    99.000000      2.000000   
75%    4.306113e+07    46.212587     6.155158   134.000000      4.000000   
max    5.069686e+07    46.346590     6.269240  3000.000000     11.000000   

         bathrooms  review_scores_rating  review_scores_location  \
count  2024.000000           1526.000000             1504.000000   
mean      1.201334              4.615577                4.782021   
std       0.497997              0.744851                0.439713   
min       0.000000              0.000000   

In [70]:
# shape of the data
print(f"Rows: {df.shape[0]}")
print(f"Columns: {df.shape[1]}")
print(f"Missing values: {df.isnull().sum().values.sum()}")
print(f"Unique values: {df.nunique()}")
print(f"Data types: {df.dtypes}")


Rows: 2024
Columns: 16
Missing values: 1018
Unique values: id                                2024
neighbourhood_cleansed              41
lat                               1677
lon                               1729
property_type                       39
room_type                            3
price                              254
accommodates                        11
bathrooms                           11
amenities                         1879
review_scores_rating               105
review_scores_location              78
instant_bookable                     2
calculated_host_listings_count      19
lake_access                          2
num_amenities                       64
dtype: int64
Data types: id                                  int64
neighbourhood_cleansed             object
lat                               float64
lon                               float64
property_type                      object
room_type                          object
price                             float6

In [71]:
# check for missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values

review_scores_rating      498
review_scores_location    520
dtype: int64

In [72]:
# replace missing values in review_scores_rating and review_scores_location with the mean
df['review_scores_rating'] = df['review_scores_rating'].fillna(df['review_scores_rating'].mean())
df['review_scores_location'] = df['review_scores_location'].fillna(df['review_scores_location'].mean())

# check for missing values or NaN values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values

Series([], dtype: int64)

In [73]:
# check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicates: {duplicates}")


Number of duplicates: 0


In [74]:
# shape of the data
print(f"Rows: {df.shape[0]}")
print(f"Columns: {df.shape[1]}")
print(f"Missing values: {df.isnull().sum().values.sum()}")
print(f"Unique values: {df.nunique()}")

Rows: 2024
Columns: 16
Missing values: 0
Unique values: id                                2024
neighbourhood_cleansed              41
lat                               1677
lon                               1729
property_type                       39
room_type                            3
price                              254
accommodates                        11
bathrooms                           11
amenities                         1879
review_scores_rating               106
review_scores_location              79
instant_bookable                     2
calculated_host_listings_count      19
lake_access                          2
num_amenities                       64
dtype: int64


In [75]:
# print summary statistics of the data
print(df.describe())


                 id          lat          lon        price  accommodates  \
count  2.024000e+03  2024.000000  2024.000000  2024.000000   2024.000000   
mean   2.992165e+07    46.206284     6.143972   126.394269      2.657115   
std    1.529775e+07     0.020470     0.027914   134.113890      1.507280   
min    4.251500e+04    46.138820     5.973220    10.000000      1.000000   
25%    1.723789e+07    46.195377     6.132483    70.000000      2.000000   
50%    3.307640e+07    46.204440     6.144230    99.000000      2.000000   
75%    4.306113e+07    46.212587     6.155158   134.000000      4.000000   
max    5.069686e+07    46.346590     6.269240  3000.000000     11.000000   

         bathrooms  review_scores_rating  review_scores_location  \
count  2024.000000           2024.000000             2024.000000   
mean      1.201334              4.615577                4.782021   
std       0.497997              0.646705                0.379010   
min       0.000000              0.000000   

### Data storage in SQLite

In [76]:
# store data in sqlite database
import sqlite3
import fnmatch

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# create a connection to the database
conn = sqlite3.connect('/workspaces/project_arm_airbnb/data/listings.db')

# Create cursor object to execute SQL commands
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

# store the data in the database
df.to_sql('listings', conn, if_exists='replace', index=False)


tourist.db
oev.db
listings.db


2024

### Query the SQL-table

In [77]:
# query the data from the database
query = "SELECT * FROM listings"
df2 = pd.read_sql(query, conn)
df2.head()


,id,neighbourhood_cleansed,lat,lon,property_type,room_type,price,accommodates,bathrooms,amenities,review_scores_rating,review_scores_location,instant_bookable,calculated_host_listings_count,lake_access,num_amenities
0,42515,Commune de Genève,46.20140,6.15507,Private room in apartment,Private room,72.0,1,1.5,"[""Cable TV"", ""Dishes and silverware"", ""Washer""...",4.69,4.79,0,1,0,22
1,44959,Commune de Genève,46.19704,6.16076,Private room in apartment,Private room,68.0,1,1.5,"[""Extra pillows and blankets"", ""Cable TV"", ""Po...",4.87,4.72,0,1,0,51
2,194110,Commune de Genève,46.20420,6.16067,Entire apartment,Entire home/apt,135.0,4,1.0,"[""Extra pillows and blankets"", ""Cable TV"", ""Di...",4.50,4.85,0,1,0,26
3,276025,Versoix,46.27944,6.16724,Entire apartment,Entire home/apt,65.0,5,1.0,"[""Extra pillows and blankets"", ""Portable fans""...",4.62,4.75,1,1,1,48
4,325986,Commune de Genève,46.21428,6.14610,Entire apartment,Entire home/apt,150.0,2,1.5,"[""Dishes and silverware"", ""Washer"", ""TV"", ""Smo...",4.84,4.76,0,1,1,26


In [78]:
# Close db connection (if open)
try:
    if check_conn(conn):
        conn.close()
    else:
        pass
except:
    pass

# Status (True = open, False = closed)
print(check_conn(conn))

False
